In [34]:
#Load required modules
import numpy as np
#from sklearn import tree
#from sklearn.metrics import accuracy_score
import pandas as pd

In [71]:
def import_data(filename):
    data = pd.read_csv(filename, sep="\t", header=0)
    return(data)

def reduce_dataset(dbl_fitness, genes, query_column = 'Query allele name', array_column = 'Array allele name'):
    query_idxs = search_genes(dbl_fitness[query_column], genes)
    print(query_idxs)
    array_idxs = search_genes(dbl_fitness[array_column], genes)
    print(array_idxs)
    idxs = intersection(query_idxs, array_idxs)
    print(idxs)
    return(dbl_fitness.ix[idxs])
    
#data = import_data('/home/dikshant/Documents/jensn lab/yeast/Data File S1. Raw genetic interaction datasets: Pair-wise interaction format/SGA_ExE.txt')
single_mutant_data = import_data('/home/dikshantpradhan/Documents/Data File S1. Raw genetic interaction datasets: Pair-wise interaction format/strain_ids_and_single_mutant_fitness.csv')
#single_mutant_data
#GO_data = import_data('/home/dikshant/GitHub/CS598-Course-Project/GOTermGeneListForClassificationYeast.txt')

training_genes = import_data('GOTermGeneListForClassificationYeastTraining.txt')
testing_genes = import_data('GOTermGeneListForClassificationYeastTest.txt')
data = pd.read_csv('specific_interaction_data.csv', sep=',',header=0)

#ExE_data = import_data('/home/dikshant/Documents/jensn lab/yeast/Data File S1. Raw genetic interaction datasets: Pair-wise interaction format/SGA_ExE.txt')
#NxN_data = import_data('/home/dikshant/Documents/jensn lab/yeast/Data File S1. Raw genetic interaction datasets: Pair-wise interaction format/SGA_NxN.txt')
#NxE_data = import_data('/home/dikshant/Documents/jensn lab/yeast/Data File S1. Raw genetic interaction datasets: Pair-wise interaction format/SGA_ExN_NxE.txt')
single_mutant_data

,Strain ID,Systematic gene name,Allele/Gene name,Single mutant fitness (26°),Single mutant fitness (26°) stddev,Single mutant fitness (30°),Single mutant fitness (30°) stddev
0,YAL001C_damp174,YAL001C,tfc3_damp,0.9395,0.0988,0.9395,0.0988
1,YAL001C_tsa508,YAL001C,tfc3-g349e,0.8658,0.1789,0.8285,0.1964
2,YAL001C_tsq508,YAL001C,tfc3-g349e,0.8658,0.1789,0.8285,0.1964
3,YAL002W_dma23,YAL002W,vps8,0.7640,0.0773,0.7640,0.0773
4,YAL002W_sn273,YAL002W,vps8,0.7640,0.0773,0.7640,0.0773
5,YAL004W_dma22,YAL004W,yal004w,1.0090,0.0227,1.0090,0.0227
6,YAL005C_dma21,YAL005C,ssa1,1.0405,0.0109,1.0405,0.0109
7,YAL007C_dma20,YAL007C,erp2,1.0260,0.0769,1.0260,0.0769
8,YAL007C_sn184,YAL007C,erp2,1.0260,0.0769,1.0260,0.0769
9,YAL008W_dma19,YAL008W,fun14,1.0445,0.0746,1.0445,0.0746


In [36]:
#all_genes = np.unique(GO_data.GeneID)
#data = reduce_dataset(data, all_genes)

data

,Unnamed: 0,X,Query.Strain.ID,Query.allele.name,Array.Strain.ID,Array.allele.name,Arraytype.Temp,Genetic.interaction.score..ε.,P.value,Query.single.mutant.fitness..SMF.,Array.SMF,Double.mutant.fitness,Double.mutant.fitness.standard.deviation
0,1,0,YAL001C_tsq508,tfc3-g349e,YBL023C_tsa111,mcm2-1,TSA30,-0.0348,5.042000e-03,0.8285,0.9254,0.7319,0.0102
1,2,7,YAL001C_tsq508,tfc3-g349e,YBL035C_tsa365,pol12-ts,TSA30,-0.0741,1.807000e-01,0.8285,0.6586,0.4715,0.0792
2,3,24,YAL001C_tsq508,tfc3-g349e,YBR029C_tsa1063,cds1-5001,TSA30,-0.1173,8.243000e-05,0.8285,0.9007,0.6289,0.0226
3,4,36,YAL001C_tsq508,tfc3-g349e,YBR087W_tsa887,rfc5-1,TSA30,-0.0663,1.788000e-01,0.8285,0.8221,0.6148,0.0550
4,5,43,YAL001C_tsq508,tfc3-g349e,YBR123C_tsa568,tfc1-e447k,TSA30,-0.5317,9.291000e-27,0.8285,0.8193,0.1470,0.0543
5,6,48,YAL001C_tsq508,tfc3-g349e,YBR154C_tsa1110,rpb5-5001,TSA30,0.1513,6.987000e-02,0.8285,0.7234,0.7506,0.1432
6,7,55,YAL001C_tsq508,tfc3-g349e,YBR193C_tsa741,med8-51,TSA30,0.1184,1.387000e-02,0.8285,0.6231,0.6346,0.0553
7,8,56,YAL001C_tsq508,tfc3-g349e,YBR193C_tsa751,med8-39,TSA30,0.1134,9.200000e-04,0.8285,0.7255,0.7145,0.0330
8,9,58,YAL001C_tsq508,tfc3-g349e,YBR198C_tsa509,taf5-3,TSA30,0.0710,1.416000e-01,0.8285,0.7784,0.7159,0.0677
9,10,59,YAL001C_tsq508,tfc3-g349e,YBR198C_tsa512,taf5-15,TSA30,0.0574,2.051000e-01,0.8285,0.6085,0.5615,0.0660


In [4]:
full_data = pd.concat([ExE_data, NxN_data, NxE_data])
full_data.index = range(full_data.shape[0])
full_data


,Query Strain ID,Query allele name,Array Strain ID,Array allele name,Arraytype/Temp,Genetic interaction score (ε),P-value,Query single mutant fitness (SMF),Array SMF,Double mutant fitness,Double mutant fitness standard deviation
0,YAL001C_tsq508,tfc3-g349e,YBL023C_tsa111,mcm2-1,TSA30,-0.0348,5.042000e-03,0.8285,0.9254,0.7319,0.0102
1,YAL001C_tsq508,tfc3-g349e,YBL026W_tsa1065,lsm2-5001,TSA30,-0.3529,3.591000e-06,0.8285,0.9408,0.4266,0.0790
2,YAL001C_tsq508,tfc3-g349e,YBL034C_tsa274,stu1-5,TSA30,0.0126,4.625000e-01,0.8285,0.8925,0.7520,0.1338
3,YAL001C_tsq508,tfc3-g349e,YBL034C_tsa454,stu1-8,TSA30,0.0043,4.998000e-01,0.8285,0.7988,0.6661,0.0831
4,YAL001C_tsq508,tfc3-g349e,YBL034C_tsa643,stu1-6,TSA30,-0.1601,5.140000e-02,0.8285,0.7683,0.4764,0.1395
5,YAL001C_tsq508,tfc3-g349e,YBL034C_tsa822,stu1-12,TSA30,0.0863,1.678000e-01,0.8285,0.9003,0.8322,0.0901
6,YAL001C_tsq508,tfc3-g349e,YBL034C_tsa950,stu1-7,TSA30,-0.1294,1.931000e-02,0.8285,0.6690,0.4249,0.0482
7,YAL001C_tsq508,tfc3-g349e,YBL035C_tsa365,pol12-ts,TSA30,-0.0741,1.807000e-01,0.8285,0.6586,0.4715,0.0792
8,YAL001C_tsq508,tfc3-g349e,YBL040C_tsa1072,erd2-5001,TSA30,0.0275,1.846000e-01,0.8285,0.6953,0.6036,0.0226
9,YAL001C_tsq508,tfc3-g349e,YBL041W_tsa1064,pre7-5001,TSA30,-0.1119,1.460000e-01,0.8285,0.8102,0.5593,0.1195


In [37]:
def parse_fitness_helper(i, j, ij, pred_tol = 0.1, class_tol = 0.1):
    pred_ij = i*j
    pred_match = np.isclose(pred_ij, ij, pred_tol)
    if (pred_match):
        return('independent')
    i_j_match = np.isclose(i,j,class_tol)
    i_ij_match = np.isclose(i,ij,class_tol)
    j_ij_match = np.isclose(j,ij,class_tol)

    if (i_j_match):
        if (j_ij_match and i_ij_match):
            return('coequal')
        elif (j < ij):
            return('synergistic')
    if (i < j):
        if (j <= ij):
            return('masking')
        elif ((i <= ij) and (ij <= j)):
            return('suppressive')
    if ((ij < i) and (ij < j)):
        return('antagonistic')

    #print(i); print(j); print(ij)
    return('unknown')

def parse_fitness(i, j, ij, pred_tol = 0.1, class_tol = 0.1):
    classification = parse_fitness_helper(i, j, ij, pred_tol, class_tol)
    if (classification == 'unknown'):
        classification = parse_fitness_helper(j, i, ij, pred_tol, class_tol)
    return(classification)

In [77]:
# build matrix
from itertools import chain

def search_list(mylist, search):
    idx = [i for i, s in enumerate(mylist) if search in s]
    return(idx)

def search_gene(mylist, gene):
    cats = ['_', '-']
    idxs = []
    for cat in cats:
        search = gene + cat
        indices = search_list(mylist, gene) #[i for i, s in enumerate(mylist) if search in s]
        idxs.append(indices)
    
    idxs = list(chain.from_iterable(idxs))
    return(idxs)

def search_genes(mylist, genes):
    idxs = []
    for gene in genes:
        new_idxs = search_gene(mylist, gene)
        idxs.append(new_idxs)
    idxs = list(chain.from_iterable(idxs))
    return(idxs)

def append_list(mylist, add):
    mylist.append(add)
    mylist = list(chain.from_iterable(mylist))
    return(mylist)

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def extract_single_mutant_fitness(df, gene, gene_column = 'Strain ID', 
                                  fitness_column = 'Single mutant fitness (26°)'):
    gene = gene#.lower()
    idxs = search_gene(df[gene_column], gene)
    mean = np.mean(df[fitness_column][idxs])
    return(mean)

#print(extract_single_mutant_fitness(single_mutant_data, 'rpc82'))

def extract_double_mutant_fitness(df, query_gene, array_gene, 
                                 query_column = 'Query.Strain.ID', array_column = 'Array.Strain.ID',
                                 fitness_column = 'Double.mutant.fitness'):
    q_idxs = search_gene(df[query_column], query_gene)
    a_idxs = search_gene(df[array_column], array_gene)
    
    dbl_mutant_idxs = intersection(q_idxs, a_idxs)
    dbl_mutant_idxs = list(np.unique(dbl_mutant_idxs))
    #print(dbl_mutant_idxs)
    #print(df[fitness_column][dbl_mutant_idxs])
    mean = np.mean(df[fitness_column][dbl_mutant_idxs])
    return(mean)
    
#extract_double_mutant_fitness(data,'tfc3','stu1')

In [74]:
reference_genes = ['TFC3', 'ARH1']
query_genes = ['MCM2', 'LSM2', 'STU1']

#reference_genes = reference_genes.lower()
#query_genes = query_genes.lower()

def build_interaction_mtx(reference_genes, query_genes, double_mutant_df, single_mutant_df):
    m = len(reference_genes)
    n = len(query_genes)
    mtx = np.zeros([n,m])
    print(reference_genes)
    
    for i in range(0,m):
        gene_i = reference_genes[i+1]#.lower()
        for j in range(0,n):
            gene_j = query_genes[j+1]#.lower()
            fitness = 0
            fitness = extract_double_mutant_fitness(double_mutant_df, gene_i, gene_j)
            mtx[j,i] = fitness
    return(mtx)

dbl_mutant_fit = build_interaction_mtx(ref, qer, data, single_mutant_data)
print(dbl_mutant_fit)
print(dbl_mutant_fit[1][0])

1    YBR019C
2    YBR020W
3    YBR229C
4    YBR299W
Name: ESTID, dtype: object
[[0.938  1.0264 0.9843    nan]
 [0.8861 0.8578 0.9014    nan]
 [0.9522 0.9699 0.9774    nan]
 [0.9715 1.0497 1.0186    nan]]
0.8861


In [84]:
def build_fitness_mtx(reference_genes, query_genes, double_mutant_df, single_mutant_df):
    m = len(reference_genes)
    n = len(query_genes)
    interaction = pd.DataFrame(index=query_genes, columns=reference_genes)
    
    double_mutant_fitness = build_interaction_mtx(reference_genes, query_genes, double_mutant_df, single_mutant_df)
    
    for i in range(0,m):
        gene_i = reference_genes[i+1]
        reference_fitness = extract_single_mutant_fitness(single_mutant_df, gene_i)
        for j in range(0,n):
            gene_j = query_genes[j+1]
            query_fitness = extract_single_mutant_fitness(single_mutant_df, gene_j)
            #print(reference_fitness); print(query_fitness)
            dbl_fitness = double_mutant_fitness[j][i]
            #print(query_fitness); print(dbl_fitness)
            interact_label = parse_fitness(reference_fitness, query_fitness, dbl_fitness)
            interaction[gene_i][gene_j] = interact_label
    
    
    return(interaction)
    # fitness = extract_single_mutant_fitness(single_mutant_df, gene_i)

mtx = build_fitness_mtx(ref, qer, data, single_mutant_data)
mtx
#mtx['MCM2']['TFC3']

1    YBR019C
2    YBR020W
3    YBR229C
4    YBR299W
Name: ESTID, dtype: object


ESTID,YBR019C,YBR020W,YBR229C,YBR299W
ESTID,,,,
YGL209W,independent,independent,independent,unknown
YGL253W,independent,independent,independent,unknown
YGR043C,independent,independent,independent,unknown
YGR143W,independent,independent,independent,unknown


In [80]:
def diff(first, second):
        second = set(second)
        return [item for item in first if item not in second]

def sample_categories(GO_data, nsamples = 5):
    categories = np.unique(GO_data.GOTermDescription)
    reference_genes = []
    query_genes = []
    for category in categories:
        idxs = np.where(GO_data.GOTermDescription == category)
        genes = GO_data.GeneID[idxs[0]]
        ref = np.random.choice(genes, size=nsamples, replace=False)
        for gene in ref:
            reference_genes.append(gene)
        #print(ref)
        query = diff(genes, ref)
        for gene in query:
            query_genes.append(gene)
        #reference_genes = reference_genes.append(ref)
        #query_genes = append_list(query_genes, query)
    reference_genes = np.unique(reference_genes)
    query_genes = np.unique(query_genes)
    return([reference_genes, query_genes])

#genes = sample_categories(GO_data)
#genes[0]

In [85]:
def interaction_dataframe(reference_genes, query_genes, double_mutant_data, single_mutant_data):
    df = pd.DataFrame(columns=reference_genes, index=query_genes)
    mtx = build_fitness_mtx(reference_genes, query_genes, double_mutant_data, single_mutant_data)
    
    for query in query_genes:
        for ref in reference_genes:
            df.loc[query][ref] = mtx.loc[query][ref]
    
    return(df)

df = interaction_dataframe(ref, qer, data, single_mutant_data)
print(df)


1    YBR019C
2    YBR020W
3    YBR229C
4    YBR299W
Name: ESTID, dtype: object
ESTID        YBR019C      YBR020W      YBR229C  YBR299W
ESTID                                                  
YGL209W  independent  independent  independent  unknown
YGL253W  independent  independent  independent  unknown
YGR043C  independent  independent  independent  unknown
YGR143W  independent  independent  independent  unknown


In [39]:
def reduce_dataset(dbl_fitness, genes, query_column = 'Query.Strain.ID', array_column = 'Array.Strain.ID'):
    query_idxs = search_genes(dbl_fitness[query_column], genes)
    array_idxs = search_genes(dbl_fitness[array_column], genes)
    
    idxs = intersection(query_idxs, array_idxs)
    
    return(dbl_fitness.ix[idxs])
    
all_genes = np.unique(GO_data.GeneID)
data = reduce_dataset(full_data, all_genes)

KeyboardInterrupt: 

In [82]:
reference_genes = testing_genes['ESTID']
query_genes = training_genes['ESTID']
ref = reference_genes[1:5]
qer = query_genes[1:5]
print(ref)
print(ref[1])

1    YBR019C
2    YBR020W
3    YBR229C
4    YBR299W
Name: ESTID, dtype: object
YBR019C


In [83]:
df = interaction_dataframe(reference_genes, query_genes, data, single_mutant_data)
#df.to_csv('full_interaction_data.csv')
df

0        YBR018C
1        YBR019C
2        YBR020W
3        YBR229C
4        YBR299W
5        YCL040W
6        YCR005C
7        YCR036W
8      YCR073W-A
9        YDL022W
10       YDL078C
11       YDR009W
12       YDR109C
13       YDR248C
14       YDR261C
15       YDR371W
16       YDR477W
17       YDR516C
18       YEL011W
19       YEL040W
20       YEL058W
21       YER003C
22       YER059W
23       YER133W
24       YFR053C
25       YGL028C
26       YGL035C
27       YGL059W
28       YGL115W
29       YGL134W
         ...    
302      YGL055W
303      YGL126W
304      YGL144C
305      YGL205W
306      YGR007W
307      YGR046W
308      YGR060W
309      YGR157W
310      YGR170W
311      YGR202C
312      YGR233C
313      YHL003C
314      YHL020C
315      YBR154C
316      YBR198C
317      YCR042C
318      YDL074C
319      YDL108W
320      YDL140C
321      YDR145W
322      YDR167W
323      YDR311W
324      YDR392W
325      YDR404C
326      YDR460W
327      YDR527W
328      YER139C
329      YER14

KeyError: 663